In [1]:
import os, requests, pandas, getpass, dotenv

dotenv.load_dotenv()

api = os.getenv('MAGIC_API') or 'https://api.earthref.org/v1/MagIC/{}'
print(api)

username = os.getenv('USER') or input()
password = os.getenv('PASS') or getpass.getpass()

### Create a Private Contribution and delete it
If no errors are reported, this cell can be repeated without any side effects. If noone has also made a private contribution between repeated executions, the same contribution ID should be reused.

In [5]:
create_response = requests.post(api.format('private'), auth=(username, password))
print(create_response.request.method, create_response.request.url)
if (create_response.status_code == 201):
    contribution_id = create_response.json()['id']
    print('Created private contribution with ID', contribution_id, '\n')

    delete_response = requests.delete(api.format('private'), params={'id': contribution_id}, auth=(username, password))
    print(delete_response.request.method, delete_response.request.url)
    if (delete_response.status_code == 200):
        print('Deleted private contribution with ID', contribution_id, '\n')
    else:
        print('Delete Private Contribution Error:', delete_response.json()['errors'][0]['message'], '\n')
else:
    print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')

POST http://localhost:3200/v1/MagIC/private
Created private contribution with ID 19470 

DELETE http://localhost:3200/v1/MagIC/private?id=19470
Deleted private contribution with ID 19470 



### Validate a Private Contribution and mark it as valid if there are no errors
The contribution ID should be in your private workspace or it will not be found.

In [7]:
contribution_id = 19470
response = requests.put(api.format('private/validate'), params={'id': contribution_id}, auth=(username, password))
print(response.request.method, response.request.url)
if (response.status_code == 200):
    validation_results = response.json()['validation']
    print('Validated contribution with ID', contribution_id, '\n', len(validation_results['errors']))
elif (response.status_code == 204):
    print('A private contribution with ID', contribution_id, 'could not be found in your private workspace for validation\n')
else:
    print('Error Validating a Private Contribution:', response.json(), '\n')

PUT http://localhost:3200/v1/MagIC/private/validate?id=19470
A private contribution with ID 19470 could not be found in your private workspace for validation



### Download a Public Contribution and create a Private Contribution to upload it to

In [10]:
contribution_id = 16901

response = requests.get(api.format('data'), params={'id': contribution_id})
print(response.request.method, response.request.url)
if (response.status_code == 200):
    contribution_file = 'downloads/magic_contribution_{}.txt'.format(contribution_id)
    open(contribution_file, 'w').write(response.text)
    print('Retrieved contribution data with ID', contribution_id, '\n')
    
    create_response = requests.post(api.format('private'), auth=(username, password))
    print(create_response.request.method, create_response.request.url)
    if (create_response.status_code == 201):
        new_contribution_id = create_response.json()['id']
        print('Created private contribution with ID', new_contribution_id, '\n')
        
        with open(contribution_file, 'rb') as f:
            upload_response = requests.put(api.format('private'), 
                params={'id': new_contribution_id}, 
                auth=(username, password), 
                headers={'Content-Type': 'text/plain'}, 
                data=f
            )

        print(upload_response.request.method, upload_response.request.url)
        if (upload_response.status_code == 202):
            print('Uploaded a text file to private contribution with ID', contribution_id, '\n')
        else:
            print('Upload Private Contribution Error:', upload_response.json()['errors'][0]['message'], '\n')
    else:
        print('Create Private Contribution Error:', create_response.json()['errors'][0]['message'], '\n')
else:
    print('Retrieve Public Contribution Error:', response.json()['errors'][0]['message'], '\n')

GET http://localhost:3200/v1/MagIC/data?id=16901
Retrieved contribution data with ID 16901 

POST http://localhost:3200/v1/MagIC/private
Created private contribution with ID 19470 

PUT http://localhost:3200/v1/MagIC/private?id=19470
Upload Private Contribution Error: must be object 

